# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-761be99fd4-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Literary Movements and physicists explorative search") 

Consider the following exploratory information need:

> investigate the literary movements and the poets. In particular, compare some physicists in terms of discovers, Nobel Prizes and education.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q43514`| theory of relativity | node |





Also consider

```
wd:Q43514 ?p ?obj .
```

is the BGP to retrieve all **properties of the theory of relativity**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Who invented the theory of relativity? (return the IRI and the label of the inventor).

2. Identify the BGP for physicist

3. Identify the BGP for Nobel Prize

4. Consider physicists born in Europe. For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

5. Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label). 

6. Amongst the physicists who won the Nobel Prize in Physics, find who had discovered or invented at least 3 things (the result set must be a list of triples physicist IRI, label and #discoveries ).

7. Amongst the physicists who won the Nobel Prize in Physics, find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).

## Task 1
Who invented the theory of relativity? (return the IRI and the label of the inventor).

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   # get the label
   ?p sc:name ?name .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
10


In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   # get the label
   ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('

In [3]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P61'), ('name', 'discoverer or inventor')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q43514 wdt:P61 ?obj .
   # get the label
   ?obj sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q937'), ('name', 'Albert Einstein')]
1


Final query for this task

In [3]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P61'), ('name', 'discoverer or inventor')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q43514 wdt:P61 ?obj .
   # get the label
   ?obj sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q937'), ('name', 'Albert Einstein')]
1


## Task 2
Identify the BGP for physicist

In [ ]:
# write your queries
# Q169470 --> physcisit
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q169470 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2163'), ('name', 'FAST ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('name', 'uses')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 's

In [5]:
# query example
# [('obj', 'http://www.wikidata.org/entity/Q937'), ('name', 'Albert Einstein')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q937 ?p ?obj .
   # get the label
   ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2861'), ('name', 'Leidse Hoogleraren ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2930'), ('name', 'INSPIRE-HEP author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6231'), ('name', 'BDELIS ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9044'), ('name', 'GEPRIS-Historisch ID (Person)')]
[('p', 'http://www.wikidata.org/prop/direct/P4116'), ('name', 'JewAge person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'ht

In [6]:
# query example
# [('obj', 'http://www.wikidata.org/entity/Q937'), ('name', 'Albert Einstein')]
# [('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q937 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19350898'), ('name', 'theoretical physicist')]
[('obj', 'http://www.wikidata.org/entity/Q16389557'), ('name', 'philosopher of science')]
[('obj', 'http://www.wikidata.org/entity/Q121594'), ('name', 'professor')]
[('obj', 'http://www.wikidata.org/entity/Q1231865'), ('name', 'pedagogue')]
[('obj', 'http://www.wikidata.org/entity/Q15980158'), ('name', 'non-fiction writer')]
[('obj', 'http://www.wikidata.org/entity/Q1622272'), ('name', 'university teacher')]
[('obj', 'http://www.wikidata.org/entity/Q169470'), ('name', 'physicist')]
[('obj', 'http://www.wikidata.org/entity/Q170790'), ('name', 'mathematician')]
[('obj', 'http://www.wikidata.org/entity/Q205375'), ('name', 'inventor')]
[('obj', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('obj', 'http://www.wikidata.org/entity/Q37226'), ('name', 'teacher')]
[('obj', 'http://www.wikidata.org/entity/Q4964182'), ('name', 'philosopher')]
[('obj', 'http://www.wikidata.org/entity/Q

In [7]:
# query example
# [('obj', 'http://www.wikidata.org/entity/Q937'), ('name', 'Albert Einstein')]
# [('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q937 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name .
   
   FILTER(regex(?name,"phys.*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19350898'), ('name', 'theoretical physicist')]
[('obj', 'http://www.wikidata.org/entity/Q169470'), ('name', 'physicist')]
2


Final query for this task

In [7]:
# query example
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q937 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name .
   
   FILTER(regex(?name,"phys.*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19350898'), ('name', 'theoretical physicist')]
[('obj', 'http://www.wikidata.org/entity/Q169470'), ('name', 'physicist')]
2


## Task 3
Identify the BGP for Nobel Prize

In [8]:
# write your queries
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q7191 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1438'), ('name', 'Jewish Encyclopedia ID (Russian)')]
[('p', 'h

In [9]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]


queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q937 wdt:P166 ?obj .
   # get the label
   OPTIONAL{
   ?obj sc:name ?name .
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1522873'), ('name', 'Josiah Willard Gibbs Lectureship')]
[('obj', 'http://www.wikidata.org/entity/Q317038'), ('name', 'Max Planck Medal')]
[('obj', 'http://www.wikidata.org/entity/Q1419938'), ('name', 'Matteucci Medal')]
[('obj', 'http://www.wikidata.org/entity/Q14906020'), ('name', 'Foreign Member of the Royal Society')]
[('obj', 'http://www.wikidata.org/entity/Q15056034'), ('name', 'Pour le Mérite for Sciences and Arts')]
[('obj', 'http://www.wikidata.org/entity/Q25287226'), ('name', 'Honorary doctor of the University of Geneva')]
[('obj', 'http://www.wikidata.org/entity/Q28003'), ('name', 'Copley Medal')]
[('obj', 'http://www.wikidata.org/entity/Q28861731'), ('name', 'honorary doctor of the Hebrew University of Jerusalem')]
[('obj', 'http://www.wikidata.org/entity/Q38104'), ('name', 'Nobel Prize in Physics')]
[('obj', 'http://www.wikidata.org/entity/Q42309226'), ('name', 'honorary doctor of the Princeton University')]
[('obj', 'http:

In [10]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
# [('obj', 'http://www.wikidata.org/entity/Q38104'), ('name', 'Nobel Prize in Physics')]

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38104 ?p ?obj .
   # get the label
   OPTIONAL{
   ?p sc:name ?name .
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3553')

In [11]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
# [('obj', 'http://www.wikidata.org/entity/Q38104'), ('name', 'Nobel Prize in Physics')]
# [('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q38104 wdt:P279 ?obj .
   # get the label
   OPTIONAL{
   ?obj sc:name ?name .
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q7191'), ('name', 'Nobel Prize')]
1


Final query for this task

In [11]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
# [('obj', 'http://www.wikidata.org/entity/Q38104'), ('name', 'Nobel Prize in Physics')]
# [('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q38104 wdt:P279 ?obj .
   # get the label
   OPTIONAL{
   ?obj sc:name ?name .
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q7191'), ('name', 'Nobel Prize')]
1


## Task 4
Consider physicists born in Europe. For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

In [12]:
# query example
#[('obj', 'http://www.wikidata.org/entity/Q169470'), ('name', 'physicist')]
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objname
WHERE {
   # bind something
   ?physicist wdt:P106 wd:Q169470;
           ?p ?obj.
           
   # get the label
   ?p sc:name ?name .
   OPTIONAL{
   ?obj sc:name ?objname
   }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P185'), ('name', 'doctoral student'), ('obj', 'http://www.wikidata.org/entity/Q102441364'), ('objname', 'Erotokritos Katsavounidis')]
[('p', 'http://www.wikidata.org/prop/direct/P184'), ('name', 'doctoral advisor'), ('obj', 'http://www.wikidata.org/entity/Q104081820'), ('objname', 'T. Kenneth Gustafson')]
[('p', 'http://www.wikidata.org/prop/direct/P184'), ('name', 'doctoral advisor'), ('obj', 'http://www.wikidata.org/entity/Q15990046'), ('objname', 'Robin M. Hochstrasser')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer'), ('obj', 'http://www.wikidata.org/entity/Q1752912'), ('objname', 'Negev Nuclear Research Center')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer'), ('obj', 'http://www.wikidata.org/entity/Q29220409'), ('objname', 'National Institute of Diabetes and Digestive and Kidney Diseases')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('name', 'member of'), ('obj', 'http

In [13]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?physicist wdt:P106 wd:Q169470;
           ?p ?obj.
           
   # get the label
   ?p sc:name ?name .
   OPTIONAL{
   ?obj sc:name ?objname
   }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2861'), ('name', 'Leidse Hoogleraren ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2930'), ('name', 'INSPIRE-HEP author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6479'), ('name', 'IEEE Xplore author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1053'), ('name', 'ResearcherID')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [14]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?physicist wdt:P106 wd:Q169470;
           wdt:P19 ?obj.
           
   # get the label
   #?p sc:name ?name .
   OPTIONAL{
   ?obj sc:name ?name
   }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q13933'), ('name', 'Ripollet')]
[('obj', 'http://www.wikidata.org/entity/Q2384286'), ('name', 'Acayucan')]
[('obj', 'http://www.wikidata.org/entity/Q19819621'), ('name', 'Duplenskaya')]
[('obj', 'http://www.wikidata.org/entity/Q4109177'), ('name', 'Verkhopenye')]
[('obj', 'http://www.wikidata.org/entity/Q345243'), ('name', 'Łapy')]
[('obj', 'http://www.wikidata.org/entity/Q20706259'), ('name', 'Voulpi')]
[('obj', 'http://www.wikidata.org/entity/Q2705385'), ('name', 'Ockham')]
[('obj', 'http://www.wikidata.org/entity/Q1023503'), ('name', 'Hudson')]
[('obj', 'http://www.wikidata.org/entity/Q587042'), ('name', 'Cluis')]
[('obj', 'http://www.wikidata.org/entity/Q156717'), ('name', 'Khmelnytskyi')]
[('obj', 'http://www.wikidata.org/entity/Q163749'), ('name', 'West Palm Beach')]
[('obj', 'http://www.wikidata.org/entity/Q206988'), ('name', 'Cheltenham')]
[('obj', 'http://www.wikidata.org/entity/Q2692715'), ('name', 'Västra Ämtervik')]
[('obj', 

In [15]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?physicist wdt:P106 wd:Q169470;
           ?p ?obj.
           
   # get the label
   ?p sc:name ?name .
   OPTIONAL{
   ?obj sc:name ?objname
   }
   
   FILTER(regex(?name,"country"))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('name', 'country of origin')]
4


In [16]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?physicist wdt:P106 wd:Q169470;
           wdt:P17 ?obj.
           
   # get the label
   #?p sc:name ?name .
   OPTIONAL{
   ?obj sc:name ?name
   }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium')]
[('obj', 'http://www.wikidata.org/entity/Q801'), ('name', 'Israel')]
[('obj', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia')]
3


In [17]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P495'), ('name', 'country of origin')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?physicist wdt:P106 wd:Q169470;
           wdt:P495 ?obj.
           
   # get the label
   #?p sc:name ?name .
   OPTIONAL{
   ?obj sc:name ?name
   }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q262'), ('name', 'Algeria')]
[('obj', 'http://www.wikidata.org/entity/Q68169763'), ('name', 'Meraldene')]
2


In [18]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?physicist wdt:P106 wd:Q169470;
           wdt:P27 ?obj.
           
   # get the label
   #?p sc:name ?name .
   OPTIONAL{
   ?obj sc:name ?name
   }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q209857'), ('name', 'Kingdom of Lombardy–Venetia')]
[('obj', 'http://www.wikidata.org/entity/Q208169'), ('name', 'Republic of Ragusa')]
[('obj', 'http://www.wikidata.org/entity/Q139319'), ('name', 'Russian Republic')]
[('obj', 'http://www.wikidata.org/entity/Q159631'), ('name', 'Kingdom of Württemberg')]
[('obj', 'http://www.wikidata.org/entity/Q704288'), ('name', 'Duchy of Holstein')]
[('obj', 'http://www.wikidata.org/entity/Q25395037'), ('name', 'Kingdom of Hungary')]
[('obj', 'http://www.wikidata.org/entity/Q330533'), ('name', 'Seventeen Provinces')]
[('obj', 'http://www.wikidata.org/entity/Q215530'), ('name', 'Kingdom of Ireland')]
[('obj', 'http://www.wikidata.org/entity/Q693911'), ('name', 'Bizone')]
[('obj', 'http://www.wikidata.org/entity/Q1377716'), ('name', 'Belgians')]
[('obj', 'http://www.wikidata.org/entity/Q223936'), ('name', 'Kingdom of Italy')]
[('obj', 'http://www.wikidata.org/entity/Q1028'), ('name', 'Morocco')]
[('obj'

In [19]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1120'), ('name', 'number of deaths')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalan

In [20]:
# query example
#(P30'), ('name', 'continent')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q38 wdt:P30 ?obj .
   # get the label
   ?obj sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
1


In [22]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
#[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
   # bind something
   ?physicist wdt:P106 wd:Q169470;
           wdt:P27 ?country.
           
   ?country wdt:P30 wd:Q46.        
           
   # get the label
   #?p sc:name ?name .
   OPTIONAL{
   ?country sc:name ?name
   }
   
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q209857'), ('name', 'Kingdom of Lombardy–Venetia')]
[('country', 'http://www.wikidata.org/entity/Q208169'), ('name', 'Republic of Ragusa')]
[('country', 'http://www.wikidata.org/entity/Q139319'), ('name', 'Russian Republic')]
[('country', 'http://www.wikidata.org/entity/Q159631'), ('name', 'Kingdom of Württemberg')]
[('country', 'http://www.wikidata.org/entity/Q704288'), ('name', 'Duchy of Holstein')]
[('country', 'http://www.wikidata.org/entity/Q25395037'), ('name', 'Kingdom of Hungary')]
[('country', 'http://www.wikidata.org/entity/Q215530'), ('name', 'Kingdom of Ireland')]
[('country', 'http://www.wikidata.org/entity/Q693911'), ('name', 'Bizone')]
[('country', 'http://www.wikidata.org/entity/Q223936'), ('name', 'Kingdom of Italy')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('name', 'German Reich')]
[('country', 'http://www.wikidata.org/entity/Q12548'), ('name', 'Holy Roman Empire')]
[('country', 'http://www.wikidata

In [28]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
#[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
   # bind something
   ?physicist wdt:P106 wd:Q169470;
           wdt:P27 ?country.
           
   ?country wdt:P30 wd:Q46.        
           
   # get the label
   OPTIONAL{
   ?country sc:name ?name
   }
   
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q209857'), ('name', 'Kingdom of Lombardy–Venetia')]
[('country', 'http://www.wikidata.org/entity/Q208169'), ('name', 'Republic of Ragusa')]
[('country', 'http://www.wikidata.org/entity/Q139319'), ('name', 'Russian Republic')]
[('country', 'http://www.wikidata.org/entity/Q159631'), ('name', 'Kingdom of Württemberg')]
[('country', 'http://www.wikidata.org/entity/Q704288'), ('name', 'Duchy of Holstein')]
[('country', 'http://www.wikidata.org/entity/Q25395037'), ('name', 'Kingdom of Hungary')]
[('country', 'http://www.wikidata.org/entity/Q215530'), ('name', 'Kingdom of Ireland')]
[('country', 'http://www.wikidata.org/entity/Q693911'), ('name', 'Bizone')]
[('country', 'http://www.wikidata.org/entity/Q223936'), ('name', 'Kingdom of Italy')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('name', 'German Reich')]
[('country', 'http://www.wikidata.org/entity/Q12548'), ('name', 'Holy Roman Empire')]
[('country', 'http://www.wikidata

In [7]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
#[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
queryString = """
SELECT DISTINCT ?physicist ?sName ?country ?name
WHERE {
   # bind something
   ?physicist wdt:P31 wd:Q5;
           wdt:P106 wd:Q169470;
           wdt:P19 ?birthplace;
           wdt:P27 ?country.
           
   ?country wdt:P31 wd:Q6256; 
           wdt:P30 wd:Q46.        
           
   # get the label
   ?physicist sc:name ?sName .
   ?country sc:name ?name .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('physicist', 'http://www.wikidata.org/entity/Q15900159'), ('sName', 'David A. B. Miller'), ('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('physicist', 'http://www.wikidata.org/entity/Q188252'), ('sName', 'Louis Michel'), ('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('physicist', 'http://www.wikidata.org/entity/Q16222639'), ('sName', 'Fernand Charron'), ('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('physicist', 'http://www.wikidata.org/entity/Q934576'), ('sName', 'Maurice Couette'), ('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('physicist', 'http://www.wikidata.org/entity/Q1386035'), ('sName', 'Savas Dimopoulos'), ('country', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece')]
[('physicist', 'http://www.wikidata.org/entity/Q132243'), ('sName', 'Rumen Ovcharov'), ('country', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria')]
[('physicist', 'ht

In [14]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
#[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
# now will count physicist who won nobel prize for each european country 
queryString = """
SELECT DISTINCT COUNT(?physicist) ?country ?name
WHERE {
   # bind something
   ?physicist wdt:P31 wd:Q5;
           wdt:P166 wd:Q38104;
           wdt:P19 ?birthplace;
           wdt:P27 ?country.
           
   ?country wdt:P31 wd:Q6256; 
           wdt:P30 wd:Q46.        
           
   # get the label
   ?physicist sc:name ?sName .
   ?country sc:name ?name .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('callret-0', '9'), ('country', 'http://www.wikidata.org/entity/Q39'), ('name', 'Switzerland')]
[('callret-0', '2'), ('country', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('callret-0', '3'), ('country', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark')]
[('callret-0', '2'), ('country', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary')]
[('callret-0', '1'), ('country', 'http://www.wikidata.org/entity/Q20'), ('name', 'Norway')]
[('callret-0', '3'), ('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('callret-0', '4'), ('country', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland')]
[('callret-0', '4'), ('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('callret-0', '27'), ('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('callret-0', '10'), ('country', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands')]
[('callret-0', '14'), ('countr

In [12]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
#[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
# now will count physicist who won nobel prize for each european country 
queryString = """
SELECT DISTINCT  ?country ?name COUNT(?physicist) AS ?physicist
WHERE {
   # bind something
   ?physicist wdt:P31 wd:Q5;
           wdt:P166 wd:Q38104;
           wdt:P19 ?birthplace;
           wdt:P27 ?country.
           
   ?country wdt:P31 wd:Q6256; 
           wdt:P30 wd:Q46.        
           
   # get the label
   ?physicist sc:name ?sName .
   ?country sc:name ?name .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q39'), ('name', 'Switzerland'), ('physicist', '9')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland'), ('physicist', '2')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark'), ('physicist', '3')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('physicist', '2')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('name', 'Norway'), ('physicist', '1')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('physicist', '3')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland'), ('physicist', '4')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('physicist', '4')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('physicist', '27')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands'), ('physicist', '10')]
[('country', 'http://www.wikid

Final query for this task

In [13]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
#[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
# now will count physicist who won nobel prize for each european country 
queryString = """
SELECT DISTINCT  ?country ?name COUNT(?physicist) AS ?physicist
WHERE {
   # bind something
   ?physicist wdt:P31 wd:Q5;
           wdt:P166 wd:Q38104;
           wdt:P19 ?birthplace;
           wdt:P27 ?country.
           
   ?country wdt:P31 wd:Q6256; 
           wdt:P30 wd:Q46.        
           
   # get the label
   ?physicist sc:name ?sName .
   ?country sc:name ?name .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q39'), ('name', 'Switzerland'), ('physicist', '9')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland'), ('physicist', '2')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark'), ('physicist', '3')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('physicist', '2')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('name', 'Norway'), ('physicist', '1')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('physicist', '3')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland'), ('physicist', '4')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('physicist', '4')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('physicist', '27')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands'), ('physicist', '10')]
[('country', 'http://www.wikid

## Task 5
Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label).

In [18]:
# write your queries
queryString = """
SELECT DISTINCT ?nobelprize ?name
WHERE {
   # bind something
   ?s wdt:P166 ?nobelprize .
   # get the label
   ?nobelprize sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('nobelprize', 'http://www.wikidata.org/entity/Q4342970'), ('name', 'Commemorative Decoration of the Second National Rising')]
[('nobelprize', 'http://www.wikidata.org/entity/Q5475056'), ('name', 'Anagrama Essay Prize')]
[('nobelprize', 'http://www.wikidata.org/entity/Q6547571'), ('name', 'Leelavati Award')]
[('nobelprize', 'http://www.wikidata.org/entity/Q10891478'), ('name', 'National Ten Outstanding Young Jurists')]
[('nobelprize', 'http://www.wikidata.org/entity/Q47337762'), ('name', 'Commander Grand Cross of the Order of the Lion of Finland')]
[('nobelprize', 'http://www.wikidata.org/entity/Q5120625'), ('name', 'CineMAA Awards')]
[('nobelprize', 'http://www.wikidata.org/entity/Q5146372'), ('name', 'College Football All-America Team')]
[('nobelprize', 'http://www.wikidata.org/entity/Q53653186'), ('name', 'Commander of the Order of the White Double Cross\u200e')]
[('nobelprize', 'http://www.wikidata.org/entity/Q62857522'), ('name', 'Honorary Member of the Chinese Academy of

In [32]:
# write your queries
queryString = """
SELECT DISTINCT  ?country ?name ?nobelprize
WHERE {
   # bind something
    ?s wdt:P166 ?nobelprize.
    ?nobelprize wdt:P279 wd:Q7191 .
           
    ?country wdt:P31 wd:Q6256.       
           
    # get the label
    ?nobelprize sc:name ?sName .
    ?country sc:name ?name .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q258'), ('name', 'South Africa'), ('nobelprize', 'http://www.wikidata.org/entity/Q35637')]
[('country', 'http://www.wikidata.org/entity/Q784'), ('name', 'Dominica'), ('nobelprize', 'http://www.wikidata.org/entity/Q35637')]
[('country', 'http://www.wikidata.org/entity/Q1049'), ('name', 'Sudan'), ('nobelprize', 'http://www.wikidata.org/entity/Q35637')]
[('country', 'http://www.wikidata.org/entity/Q902'), ('name', 'Bangladesh'), ('nobelprize', 'http://www.wikidata.org/entity/Q35637')]
[('country', 'http://www.wikidata.org/entity/Q657'), ('name', 'Chad'), ('nobelprize', 'http://www.wikidata.org/entity/Q35637')]
[('country', 'http://www.wikidata.org/entity/Q1019'), ('name', 'Madagascar'), ('nobelprize', 'http://www.wikidata.org/entity/Q35637')]
[('country', 'http://www.wikidata.org/entity/Q836'), ('name', 'Myanmar'), ('nobelprize', 'http://www.wikidata.org/entity/Q35637')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('name', 'Pola

In [9]:
# write your queries
# Nobel Prize (Q7191)
# country (Q6256)
queryString = """
SELECT DISTINCT  ?country ?countryName
WHERE {
   # bind something
    ?s wdt:P166 ?nobelprize;
        wdt:P19 ?birthPlace.
        
    ?birthPlace wdt:P17 ?country.
    ?nobelprize wdt:P279 wd:Q7191 .
           
    ?country wdt:P31 wd:Q6256.       
           
    # get the label
    ?country sc:name ?countryName .
   
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm')]
[('country', 'http://www.wikidata.org/entity/Q1014'), ('countryName', 'Liberia')]
[('country', 'http://www.wikidata.org/entity/Q1019'), ('countryName', 'Madagascar')]
[('country', 'http://www.wikidata.org/entity/Q1028'), ('countryName', 'Morocco')]
[('country', 'http://www.wikidata.org/entity/Q1033'), ('countryName', 'Nigeria')]
[('country', 'http://www.wikidata.org/entity/Q114'), ('countryName', 'Kenya')]
[('country', 'http://www.wikidata.org/entity/Q115'), ('countryName', 'Ethiopia')]
[('country', 'http://www.wikidata.org/entity/Q117'), ('countryName', 'Ghana')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('countryName', "People's Republic of China")]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryNa

Final query for this task

In [9]:
# write your final query
# Nobel Prize (Q7191)
# country (Q6256)
queryString = """
SELECT DISTINCT  ?country ?countryName
WHERE {
   # bind something
    ?s wdt:P166 ?nobelprize;
        wdt:P19 ?birthPlace.
        
    ?birthPlace wdt:P17 ?country.
    ?nobelprize wdt:P279 wd:Q7191 .
           
    ?country wdt:P31 wd:Q6256.       
           
    # get the label
    ?country sc:name ?countryName .
   
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm')]
[('country', 'http://www.wikidata.org/entity/Q1014'), ('countryName', 'Liberia')]
[('country', 'http://www.wikidata.org/entity/Q1019'), ('countryName', 'Madagascar')]
[('country', 'http://www.wikidata.org/entity/Q1028'), ('countryName', 'Morocco')]
[('country', 'http://www.wikidata.org/entity/Q1033'), ('countryName', 'Nigeria')]
[('country', 'http://www.wikidata.org/entity/Q114'), ('countryName', 'Kenya')]
[('country', 'http://www.wikidata.org/entity/Q115'), ('countryName', 'Ethiopia')]
[('country', 'http://www.wikidata.org/entity/Q117'), ('countryName', 'Ghana')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('countryName', "People's Republic of China")]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryNa

## Task 6
Amongst the physicists who won the Nobel Prize in Physics, find who had discovered or invented at least 3 things (the result set must be a list of triples physicist IRI, label and #discoveries ).

In [36]:
# write your queries
queryString = """
SELECT DISTINCT ?physicist ?sName ?discover ?discoverName
WHERE {
   ?physicist wdt:P166 wd:Q38104 .  #who received nobel price
   ?discover wdt:P61 ?physicist .   #inventor
   
   #get the label
   ?physicist sc:name ?sName .
   ?discover sc:name ?discoverName .
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('physicist', 'http://www.wikidata.org/entity/Q71024'), ('sName', 'Klaus von Klitzing'), ('discover', 'http://www.wikidata.org/entity/Q1047822'), ('discoverName', 'quantum Hall effect')]
[('physicist', 'http://www.wikidata.org/entity/Q71022'), ('sName', 'Ernst Ruska'), ('discover', 'http://www.wikidata.org/entity/Q132560'), ('discoverName', 'electron microscope')]
[('physicist', 'http://www.wikidata.org/entity/Q191922'), ('sName', 'Frederick Reines'), ('discover', 'http://www.wikidata.org/entity/Q1066748'), ('discoverName', 'antineutrino')]
[('physicist', 'http://www.wikidata.org/entity/Q57068'), ('sName', 'Wilhelm Wien'), ('discover', 'http://www.wikidata.org/entity/Q214336'), ('discoverName', "Wien's displacement law")]
[('physicist', 'http://www.wikidata.org/entity/Q57077'), ('sName', 'Ferdinand Braun'), ('discover', 'http://www.wikidata.org/entity/Q83913'), ('discoverName', 'cathode ray tube')]
[('physicist', 'http://www.wikidata.org/entity/Q57077'), ('sName', 'Ferdinand B

In [37]:
# write your queries
queryString = """
SELECT DISTINCT ?physicist ?sName (COUNT(?discover) as ?totalNoOfDiscoveries)
WHERE {
   ?physicist wdt:P166 wd:Q38104 .  #who received nobel price
   ?discover wdt:P61 ?physicist .   #inventor
   
   #get the label
   ?physicist sc:name ?sName .
   ?discover sc:name ?discoverName .
}
GROUP BY ?physicist ?sName
HAVING (COUNT(?disc) >=3)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [40]:
# write your queries
queryString = """
SELECT DISTINCT ?physicist ?sName (COUNT(?discover) as ?totalNoOfDiscoveries)
WHERE {
   ?physicist wdt:P166 wd:Q38104 .  #who received nobel price
   ?discover wdt:P61 ?physicist .   #inventor
   
   #get the label
   ?physicist sc:name ?sName .
   ?discover sc:name ?discoverName .
}
GROUP BY ?physicist ?sName
HAVING (COUNT(?discover) >2)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('physicist', 'http://www.wikidata.org/entity/Q193803'), ('sName', 'Roger Penrose'), ('totalNoOfDiscoveries', '6')]
[('physicist', 'http://www.wikidata.org/entity/Q160518'), ('sName', 'Gustaf Dalén'), ('totalNoOfDiscoveries', '3')]
[('physicist', 'http://www.wikidata.org/entity/Q8753'), ('sName', 'Enrico Fermi'), ('totalNoOfDiscoveries', '6')]
[('physicist', 'http://www.wikidata.org/entity/Q123975'), ('sName', 'Michel Mayor'), ('totalNoOfDiscoveries', '12')]
[('physicist', 'http://www.wikidata.org/entity/Q173028'), ('sName', 'Emilio G. Segrè'), ('totalNoOfDiscoveries', '3')]
[('physicist', 'http://www.wikidata.org/entity/Q124013'), ('sName', 'Didier Queloz'), ('totalNoOfDiscoveries', '3')]
[('physicist', 'http://www.wikidata.org/entity/Q47480'), ('sName', 'Paul Dirac'), ('totalNoOfDiscoveries', '3')]
[('physicist', 'http://www.wikidata.org/entity/Q937'), ('sName', 'Albert Einstein'), ('totalNoOfDiscoveries', '10')]
[('physicist', 'http://www.wikidata.org/entity/Q37463'), ('sNa

Final query for this task

In [43]:
# write your queries
queryString = """
SELECT DISTINCT ?physicist ?sName (COUNT(?discover) as ?totalNoOfDiscoveries)
WHERE {
   ?physicist wdt:P166 wd:Q38104 .  #who received nobel price
   ?discover wdt:P61 ?physicist .   #inventor
   
   #get the label
   ?physicist sc:name ?sName .
   ?discover sc:name ?discoverName .
}
GROUP BY ?physicist ?sName
HAVING (COUNT(?discover) >2)   #more then or equal 3
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('physicist', 'http://www.wikidata.org/entity/Q193803'), ('sName', 'Roger Penrose'), ('totalNoOfDiscoveries', '6')]
[('physicist', 'http://www.wikidata.org/entity/Q160518'), ('sName', 'Gustaf Dalén'), ('totalNoOfDiscoveries', '3')]
[('physicist', 'http://www.wikidata.org/entity/Q8753'), ('sName', 'Enrico Fermi'), ('totalNoOfDiscoveries', '6')]
[('physicist', 'http://www.wikidata.org/entity/Q123975'), ('sName', 'Michel Mayor'), ('totalNoOfDiscoveries', '12')]
[('physicist', 'http://www.wikidata.org/entity/Q173028'), ('sName', 'Emilio G. Segrè'), ('totalNoOfDiscoveries', '3')]
[('physicist', 'http://www.wikidata.org/entity/Q124013'), ('sName', 'Didier Queloz'), ('totalNoOfDiscoveries', '3')]
[('physicist', 'http://www.wikidata.org/entity/Q47480'), ('sName', 'Paul Dirac'), ('totalNoOfDiscoveries', '3')]
[('physicist', 'http://www.wikidata.org/entity/Q937'), ('sName', 'Albert Einstein'), ('totalNoOfDiscoveries', '10')]
[('physicist', 'http://www.wikidata.org/entity/Q37463'), ('sNa

## Task 7
Amongst the physicists who won the Nobel Prize in Physics, find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).

In [2]:
# write your queries
queryString = """
SELECT DISTINCT ?physicist ?sName ?discover ?discoverName
WHERE {
   ?physicist wdt:P166 wd:Q38104 .  #who received nobel price
   ?discover wdt:P61 ?physicist .   #inventor
   
   #get the label
   ?physicist sc:name ?sName .
   ?discover sc:name ?discoverName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('physicist', 'http://www.wikidata.org/entity/Q71024'), ('sName', 'Klaus von Klitzing'), ('discover', 'http://www.wikidata.org/entity/Q1047822'), ('discoverName', 'quantum Hall effect')]
[('physicist', 'http://www.wikidata.org/entity/Q71022'), ('sName', 'Ernst Ruska'), ('discover', 'http://www.wikidata.org/entity/Q132560'), ('discoverName', 'electron microscope')]
[('physicist', 'http://www.wikidata.org/entity/Q191922'), ('sName', 'Frederick Reines'), ('discover', 'http://www.wikidata.org/entity/Q1066748'), ('discoverName', 'antineutrino')]
[('physicist', 'http://www.wikidata.org/entity/Q57068'), ('sName', 'Wilhelm Wien'), ('discover', 'http://www.wikidata.org/entity/Q214336'), ('discoverName', "Wien's displacement law")]
[('physicist', 'http://www.wikidata.org/entity/Q57077'), ('sName', 'Ferdinand Braun'), ('discover', 'http://www.wikidata.org/entity/Q83913'), ('discoverName', 'cathode ray tube')]
[('physicist', 'http://www.wikidata.org/entity/Q57077'), ('sName', 'Ferdinand B

In [11]:
# write your queries
queryString = """
SELECT DISTINCT ?physicist ?sName ?uni ?uniName
WHERE {
   ?physicist wdt:P166 wd:Q38104;   #who received nobel price
        wdt:P106 wd:Q1622272;  #Occupation is uni teacher
        wdt:P108 ?uni.  #uni employee
   
   
   #get the label
   ?physicist sc:name ?sName .
   ?uni sc:name ?uniName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('physicist', 'http://www.wikidata.org/entity/Q57072'), ('sName', 'Otto Stern'), ('uni', 'http://www.wikidata.org/entity/Q159895'), ('uniName', 'University of Rostock')]
[('physicist', 'http://www.wikidata.org/entity/Q57072'), ('sName', 'Otto Stern'), ('uni', 'http://www.wikidata.org/entity/Q190080'), ('uniName', 'Carnegie Mellon University')]
[('physicist', 'http://www.wikidata.org/entity/Q57072'), ('sName', 'Otto Stern'), ('uni', 'http://www.wikidata.org/entity/Q168756'), ('uniName', 'University of California, Berkeley')]
[('physicist', 'http://www.wikidata.org/entity/Q57072'), ('sName', 'Otto Stern'), ('uni', 'http://www.wikidata.org/entity/Q156725'), ('uniName', 'University of Hamburg')]
[('physicist', 'http://www.wikidata.org/entity/Q57072'), ('sName', 'Otto Stern'), ('uni', 'http://www.wikidata.org/entity/Q11942'), ('uniName', 'ETH Zürich')]
[('physicist', 'http://www.wikidata.org/entity/Q71024'), ('sName', 'Klaus von Klitzing'), ('uni', 'http://www.wikidata.org/entity/Q

In [12]:
# write your queries
queryString = """
SELECT DISTINCT ?physicist ?sName ?uni ?uniName
WHERE {
   ?physicist wdt:P166 wd:Q38104;   #who received nobel price
        wdt:P106 wd:Q1622272;  #Occupation is uni teacher
        wdt:P108 ?uni;  #uni employee
        wdt:P19 ?birthPlace.
   
   
   #get the label
   ?physicist sc:name ?sName .
   ?uni sc:name ?uniName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('physicist', 'http://www.wikidata.org/entity/Q133747'), ('sName', 'William Henry Bragg'), ('uni', 'http://www.wikidata.org/entity/Q15574'), ('uniName', 'University of Adelaide')]
[('physicist', 'http://www.wikidata.org/entity/Q181363'), ('sName', 'Brian David Josephson'), ('uni', 'http://www.wikidata.org/entity/Q4380853'), ('uniName', 'Missouri University of Science and Technology')]
[('physicist', 'http://www.wikidata.org/entity/Q41269'), ('sName', 'Henri Becquerel'), ('uni', 'http://www.wikidata.org/entity/Q838691'), ('uniName', "Muséum national d'histoire naturelle")]
[('physicist', 'http://www.wikidata.org/entity/Q71013'), ('sName', 'Theodor W. Hänsch'), ('uni', 'http://www.wikidata.org/entity/Q829535'), ('uniName', 'Max Planck Institute of Quantum Optics')]
[('physicist', 'http://www.wikidata.org/entity/Q57100'), ('sName', 'Maria Goeppert Mayer'), ('uni', 'http://www.wikidata.org/entity/Q649120'), ('uniName', 'Argonne National Laboratory')]
[('physicist', 'http://www.wik

In [3]:
# write your queries
#I'll try to retrieve the place of the sapienza uni
#  ('uni', 'http://www.wikidata.org/entity/Q209344'), ('uniName', 'Sapienza University of Rome')]
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q209344 ?p ?obj .
   
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location'), ('obj', 'http://www.wikidata.org/entity/Q102114571'), ('objName', 'città universitaria of Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q241791'), ('objName', 'Bracciano')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q116592'), ('objName', 'Cassino')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q128226'), ('objName', 'Terracina')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q242533'), ('objName', 'Civitavecchia')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q271411'), ('objName', 'Malindi')]
[('p', 'http://www.wikidata.org/prop/direct/P2

In [11]:
# write your queries
#I'll try to retrieve the place of the sapienza uni
#  ('uni', 'http://www.wikidata.org/entity/Q209344'), ('uniName', 'Sapienza University of Rome')]
queryString = """
SELECT DISTINCT ?physicist ?sName ?uni ?uniName ?birthPlace ?birthplaceName
WHERE {
   ?physicist wdt:P166 wd:Q38104;   #who received nobel price
        wdt:P106 wd:Q1622272;  #Occupation is uni teacher
        wdt:P108 ?uni;  #uni employee
        wdt:P19 ?birthPlace.
   
   ?birthPlace wdt:P31 wd:Q515 .
   ?uni ?p ?birthPlace .
   #get the label
   ?physicist sc:name ?sName .
   ?uni sc:name ?uniName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('physicist', 'http://www.wikidata.org/entity/Q44286'), ('sName', 'Rudolf Mössbauer'), ('uni', 'http://www.wikidata.org/entity/Q157808'), ('uniName', 'Technical University of Munich'), ('birthPlace', 'http://www.wikidata.org/entity/Q1726')]
[('physicist', 'http://www.wikidata.org/entity/Q184499'), ('sName', 'Patrick Blackett, Baron Blackett'), ('uni', 'http://www.wikidata.org/entity/Q375606'), ('uniName', 'Birkbeck, University of London'), ('birthPlace', 'http://www.wikidata.org/entity/Q84')]
2


In [3]:
# write your queries
#I'll try to retrieve the place of the sapienza uni
#  ('uni', 'http://www.wikidata.org/entity/Q209344'), ('uniName', 'Sapienza University of Rome')]
# ('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location')
queryString = """
SELECT DISTINCT ?physicist ?sName ?uni ?uniName ?birthPlace ?birthplaceName
WHERE {
   ?physicist wdt:P166 wd:Q38104;   #who received nobel price
        wdt:P106 wd:Q1622272;  #Occupation is uni teacher
        wdt:P108 ?uni;  #uni employee
        wdt:P19 ?birthPlace.
   
   ?birthPlace wdt:P31 wd:Q515 .
   {
       ?uni ?p ?birthPlace .
   }
   UNION{
       ?university wdt:P159 ?headquarter .
       ?headquarter ?p1 ?birthPlace .
   }
   
   #get the label
   ?physicist sc:name ?sName .
   ?p sc:name ?pName .
   ?uni sc:name ?uniName .
   ?birthPlace sc:name ?birthplaceName
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out


Final query for this task

In [4]:
# write your final query
#I'll try to retrieve the place of the sapienza uni
# ('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location')
# city (Q515)
queryString = """
SELECT DISTINCT ?physicist ?sName ?uni ?uniName
WHERE {
   ?physicist wdt:P166 wd:Q38104;   #who received nobel price
        wdt:P106 wd:Q1622272;  #Occupation is uni teacher
        wdt:P108 ?uni;  #uni employee
        wdt:P19 ?birthPlace.
   
   ?birthPlace wdt:P31 wd:Q515 .
   {
       ?uni ?p ?birthPlace .
   }
   UNION{
       ?university wdt:P159 ?headquarter .
       ?headquarter ?p1 ?birthPlace .
   }
   
   #get the label
   ?physicist sc:name ?sName .
   ?p sc:name ?pName .
   ?uni sc:name ?uniName .
   ?birthPlace sc:name ?birthplaceName
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out
